In [3]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests

In [10]:


url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/115.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.raise_for_status()

html_content = response.text

# Parse all tables and pick the one that contains a Symbol (or Ticker) column.
tables = pd.read_html(html_content)
df_sp = None
for tbl in tables:
    cols = [str(c) for c in tbl.columns]
    if any(c.strip() == 'Symbol' for c in cols) or any('Symbol' in c for c in cols):
        df_sp = tbl
        break
if df_sp is None:
    # fallback: look for common alternatives like 'Ticker'
    for tbl in tables:
        cols = [str(c) for c in tbl.columns]
        if any('Ticker' in c for c in cols):
            df_sp = tbl
            break
if df_sp is None:
    raise KeyError("Could not find a table with a 'Symbol' or 'Ticker' column on the page")

# Normalize column name and convert tickers for yfinance (replace '.' with '-')
symbol_col = next((c for c in df_sp.columns if 'Symbol' in str(c) or 'Ticker' in str(c)), None)
tickers = df_sp[symbol_col].astype(str).str.replace('.', '-', regex=False).tolist()

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\3469017320.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_content)


In [11]:
df= yf.download(tickers, start='2020-01-01')['Close']

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\532441637.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df= yf.download(tickers, start='2020-01-01')['Close']
[*********************100%***********************]  503 of 503 completed


In [19]:
df_ret = df.pct_change().fillna(0)
df_ret_corr = df_ret.corr()
df_ret_corr

C:\Users\simin\AppData\Local\Temp\ipykernel_18144\612118434.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ret = df.pct_change().fillna(0)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Ticker,,,,,,,,,,,,,,,,,,,,,
A,1.000000,0.499866,0.381276,0.290337,0.510884,0.365397,0.556347,0.479077,0.560011,0.348873,...,0.524630,0.361043,0.332506,0.303988,0.550448,0.467940,0.404857,0.408767,0.562624,0.575683
AAPL,0.499866,1.000000,0.292551,0.341697,0.428518,0.347490,0.566230,0.599166,0.599724,0.312884,...,0.498960,0.372501,0.355604,0.293033,0.493025,0.507117,0.418288,0.357145,0.543943,0.509235
ABBV,0.381276,0.292551,1.000000,0.033413,0.433202,0.365405,0.344000,0.247705,0.288403,0.292225,...,0.362830,0.199408,0.344163,0.307222,0.315915,0.164722,0.336561,0.349619,0.235013,0.398643
ABNB,0.290337,0.341697,0.033413,1.000000,0.136251,0.105793,0.298397,0.342319,0.382572,0.109059,...,0.215333,0.323935,-0.009784,0.146349,0.272533,0.426717,0.194475,0.182454,0.361728,0.243068
ABT,0.510884,0.428518,0.433202,0.136251,1.000000,0.382735,0.484879,0.402328,0.400421,0.333420,...,0.454468,0.188265,0.478255,0.214142,0.450875,0.336805,0.408656,0.415611,0.368606,0.539429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYZ,0.467940,0.507117,0.164722,0.426717,0.336805,0.257254,0.472048,0.525569,0.516804,0.201263,...,0.442644,0.401122,0.224308,0.212602,0.428359,1.000000,0.347200,0.325115,0.511927,0.424657
YUM,0.404857,0.418288,0.336561,0.194475,0.408656,0.478148,0.475703,0.321790,0.427569,0.371083,...,0.559421,0.439409,0.411531,0.371626,0.488174,0.347200,1.000000,0.496010,0.364368,0.451684
ZBH,0.408767,0.357145,0.349619,0.182454,0.415611,0.440418,0.440678,0.279744,0.402090,0.392404,...,0.495026,0.445872,0.304081,0.410523,0.455449,0.325115,0.496010,1.000000,0.370121,0.413044


In [ ]:
pairwise_dist = ((1 - df_ret_corr) / 2).pow(2)
pairwise_dist